# [SpaceNet](https://aws.amazon.com/public-datasets/spacenet/)

"The current SpaceNet corpus includes **thousands of square kilometers of high resolution imagery** collected from **DigitalGlobe’s commercial satellites** which includes **8-band multispectral data**. This dataset is being made public to advance the development of **algorithms to automatically extract geometric features such as roads, building footprints, and points of interest using satellite imagery**. The currently available Areas of Interest (AOI) are **Rio De Janeiro**, Paris, Las Vegas, Shanghai and Khartoum."

### 0. Dependencies
The [AWS Command Line Interface (CLI)](https://aws.amazon.com/cli/) must be installed with an active AWS account. Configure the AWS CLI using `aws configure`.

### 1. Accessing the SpaceNet Data on AWS
The imagery is [GeoTIFF](https://en.wikipedia.org/wiki/GeoTIFF) satellite imagery and corresponding [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) building footprints.

The spacenet-dataset S3 bucket is provided as a [Requester Pays bucket](https://docs.aws.amazon.com/AmazonS3/latest/dev/RequesterPaysBuckets.html), so we use [Boto](https://boto3.readthedocs.io/en/latest/index.html), the Amazon Web Services (AWS) SDK for Python.

In [2]:
import boto3
client = boto3.client("s3")
# https://boto3.readthedocs.io/en/latest/reference/customizations/s3.html#boto3.s3.transfer.S3Transfer
transfer = boto3.s3.transfer.S3Transfer(client)

bucket = "spacenet-dataset"
# 20 tiffs listed in "AOI_1_Rio_manifest.txt".
names = [
    "013022223310.tif",
    "013022232021.tif",
    "013022232201.tif",
    "013022223131.tif",
    "013022223113.tif",
    "013022223103.tif",
    "013022223133.tif",
    "013022223132.tif",
    "013022223301.tif",
    "013022223112.tif",
    "013022232020.tif",
    "013022232200.tif",
    "013022223123.tif",
    "013022232022.tif",
    "013022223130.tif",
    "013022232002.tif",
    "013022232023.tif",
    "013022223311.tif",
    "013022232003.tif",
    "013022223121.tif"
]

key_prefix = "AOI_1_Rio/srcData/mosaic_3band/"
!mkdir -p /tmp/spacenet-data
filename_prefix = "/tmp/spacenet-data/"

key_filename_tuples = [
    (key_prefix + name, filename_prefix + name)
    for name in names]

# Download takes 4 minutes for 3-band, 16 minutes for 8-band.
import time
start = time.time()

for (key, filename) in key_filename_tuples:
    transfer.download_file(
        bucket=bucket, key=key, filename=filename,
        extra_args={"RequestPayer":"requester"}
    )

end = time.time()
download_time = end - start
minutes = int(download_time)/60
print("Download time: %d minutes" % int(minutes))

Download time: 5 minutes


## 2. Ingest Images with GeoPySpark

[GeoPySpark](https://github.com/locationtech-labs/geopyspark) is a Python language binding library of the Scala library, [GeoTrellis](https://github.com/locationtech/geotrellis), which reads, writes, and operates on raster data as fast as possible using Spark.

Refer to [Ingesting a Grayscale Image](https://geopyspark.readthedocs.io/en/latest/tutorials/greyscale_ingest_example.html) tutorial for code breakdown.

In [3]:
from pyspark import SparkContext
from geopyspark import geopyspark_conf
conf = geopyspark_conf("local[*]", "spacenet-ingest")
geopysc = SparkContext.getOrCreate(conf)

In [4]:
# Ingest takes X minutes.
import time
start = time.time()

from geopyspark.geotrellis.geotiff import get
from geopyspark.geotrellis.constants import SPATIAL, ZOOM
from geopyspark.geotrellis.catalog import write

# Read the GeoTiff locally
rdd = get(geopysc, SPATIAL, "file:///tmp/spacenet-data/")
# Error: https://github.com/locationtech/geotrellis/issues/2268
metadata = rdd.collect_metadata()

# tile the rdd to the layout defined in the metadata
laid_out = rdd.tile_to_layout(metadata)

# reproject the tiled rasters using a ZoomedLayoutScheme
reprojected = laid_out.reproject("EPSG:3857", scheme=ZOOM)#.cache().repartition(200)

# pyramid the TiledRasterRDD to create 12 new TiledRasterRDDs
# one for each zoom level
pyramided = reprojected.pyramid(start_zoom=12, end_zoom=1)

# Save each TiledRasterRDD locally
for tiled in pyramided:
    write("file:///tmp/spacenet-catalog", "spacenet-ingest", tiled)

end = time.time()
ingest_time = end - start
minutes = int(ingest_time)/60
print("Ingest time: %d minutes" % minutes)

Py4JJavaError: An error occurred while calling o29.collectMetadata.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 0.0 failed 1 times, most recent failure: Lost task 1.0 in stage 0.0 (TID 1, localhost, executor driver): scala.MatchError: (53971,53970) (of class scala.Tuple2$mcII$sp)
	at geotrellis.raster.io.geotiff.reader.TiffTagsReader$.readTag(TiffTagsReader.scala:114)
	at geotrellis.raster.io.geotiff.reader.TiffTagsReader$.read(TiffTagsReader.scala:102)
	at geotrellis.raster.io.geotiff.reader.GeoTiffReader$.readGeoTiffInfo(GeoTiffReader.scala:288)
	at geotrellis.raster.io.geotiff.reader.GeoTiffReader$.readMultiband(GeoTiffReader.scala:174)
	at geotrellis.raster.io.geotiff.reader.GeoTiffReader$.readMultiband(GeoTiffReader.scala:152)
	at geotrellis.raster.io.geotiff.MultibandGeoTiff$.apply(MultibandGeoTiff.scala:93)
	at geotrellis.spark.io.RasterReader$$anon$2.readFully(RasterReader.scala:98)
	at geotrellis.spark.io.RasterReader$$anon$2.readFully(